# pygmsh testing

In [1]:
%matplotlib widget

In [2]:
# from bmcs_shell.folding.wb_model import WBModel
# from bmcs_shell.folding.vmats2D_elastic import MATS2DElastic
# import bmcs_utils.api as bu
import pygmsh
import numpy as np
import k3d

In [3]:
X_Id = np.array([[0,0,0], [1,0,0], [1,1,0], [1, 0, 1]])
I_Fi = np.array([[0,1,2], [0,1,3]])
mesh_size = 2 * np.linalg.norm(X_Id[1]-X_Id[0])

X_Fid = X_Id[I_Fi]
with pygmsh.geo.Geometry() as geom:
    for X_id in X_Fid:
        geom.add_polygon(X_id, mesh_size=3)
    mesh = geom.generate_mesh()

X_Id_mesh = mesh.points
I_Fi_mesh = mesh.cells[1][1]

k3d_mesh = k3d.mesh(X_Id_mesh, I_Fi_mesh, side='double', color=0x75ade6)
k3d_mesh += k3d.mesh(X_Id_mesh, I_Fi_mesh, side='double', color=0x000000, wireframe=True)
k3d_mesh

C:\Users\Homam\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
C:\Users\Homam\Anaconda3\envs\bmcs_env\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "uint64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [191]:
mesh

<meshio mesh object>
  Number of points: 6
  Number of cells:
    line: 6
    triangle: 2
    vertex: 6

**Note:** From the output of the last cell we see that pygmsh doesn't automatically merges the overlapping mesh (6 lines instead of 5)

## Obtaining lines mapping from facets mapping `I_Fi`

In [101]:
I_Fi

array([[0, 1, 2],
       [0, 1, 3]])

In [102]:
lines1 = I_Fi[:, [0, 1]]
lines2 = I_Fi[:, [0, 2]]
lines3 = I_Fi[:, [1, 2]]
lines = np.vstack((lines1, lines2, lines3))
lines

array([[0, 1],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3]])

In [103]:
lines = np.sort(lines,axis=1)
lines

array([[0, 1],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3]])

In [104]:
lines = np.unique(a, axis=0)
lines

array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 2],
       [1, 3]])

### All together

In [7]:
def get_lines(I_Fi):
    lines1 = I_Fi[:, [0, 1]]
    lines2 = I_Fi[:, [0, 2]]
    lines3 = I_Fi[:, [1, 2]]
    lines = np.vstack((lines1, lines2, lines3))
    lines = np.sort(lines,axis=1)
    lines = np.unique(lines, axis=0)
    return lines

In [74]:
import gmsh
import sys
import numpy as np

X_Id = np.array([[0,0,0], [1,0,0], [1,1,0], [1, 0, 1]])
I_Fi = np.array([[0,1,2], [0,1,3]])

gmsh.initialize()
gmsh.model.add("t1")

mesh_size = np.full(X_Id.shape[0], 0.1)

# TODO here, I'm taking only one surface, take all
points_coords = X_Id[I_Fi][0]

points = [gmsh.model.geo.add_point(*x, l) for x, l in zip(points_coords, mesh_size)]
curves = [gmsh.model.geo.add_line(points[k], points[k + 1]) for k in range(len(points) - 1)] + [gmsh.model.geo.add_line(points[-1], points[0])]

curve_loop = gmsh.model.geo.add_curve_loop(curves)
surface = gmsh.model.geo.add_plane_surface([curve_loop])

gmsh.model.geo.synchronize()

# Adding BCs
gmsh.model.add_physical_group(1, [1, 2], 555)
gmsh.model.set_physical_name(2, 555, "Fixed support")

gmsh.model.mesh.generate(3)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

In [65]:
# Old test
import gmsh
import sys

X_Id = np.array([[0,0,0], [1,0,0], [1,1,0]])
I_Fi = np.array([[0,1,2]])

gmsh.initialize()
gmsh.model.add("t1")

X_Fid = X_Id[I_Fi]
for point in X_Id:
    gmsh.model.geo.add_point(*point, 0.1)
    
# adding one because default points' tags start with 1
lines = get_lines(I_Fi) + 1
print(lines)
for line in lines:
    gmsh.model.geo.add_line(*line)

gmsh.model.geo.add_curve_loop([1, -2, 3])
gmsh.model.geo.add_plane_surface([1])
gmsh.model.geo.synchronize()

gmsh.model.mesh.generate(3)

if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

[[1 2]
 [1 3]
 [2 3]]


## Test with the wb shell

In [4]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=2
n_x_plus=1
wbm = WBModel(F=-20000, h=10, tmodel=tmodel)
data = dict(alpha=1., a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=True)
wbm.wb_mesh.trait_set(**data);

In [5]:
wbm.app()

In [5]:
wbm.wb_mesh.X_Id  # nodal coordinates

array([[    0.        ,   808.02069092,   -59.7734375 ],
       [  500.        ,  1696.42700195,    84.22070312],
       [ -500.        ,  1696.42700195,    84.22070312],
       [  500.        ,     0.        ,   336.58886719],
       [ -500.        ,     0.        ,   336.58886719],
       [  216.1209259 ,   857.54797363,   273.15136719],
       [ -216.1209259 ,   857.54797363,   273.15136719],
       [    0.        ,  -808.02069092,   -59.7734375 ],
       [  500.        , -1696.42700195,    84.22070312],
       [ -500.        , -1696.42700195,    84.22070312],
       [  216.1209259 ,  -857.54797363,   273.15136719],
       [ -216.1209259 ,  -857.54797363,   273.15136719]])

In [6]:
wbm.wb_mesh.I_Fi  # elements

array([[ 0,  1,  2],
       [ 0,  3,  4],
       [ 0,  1,  5],
       [ 0,  2,  6],
       [ 0,  3,  5],
       [ 0,  4,  6],
       [ 7,  3,  4],
       [ 7,  8,  9],
       [ 7,  3, 10],
       [ 7,  4, 11],
       [ 7,  8, 10],
       [ 7,  9, 11]])

In [17]:
import numpy as np

U_1 = wbm.hist.U_t[-1]
# U_max = np.max(np.fabs(U_1))
# U_max
U_1 
U_1.reshape(-1, 3) * 1
# _, loaded_nodes, _ = wbm.bc_loaded
# wbm.xdomain.I_Ei.astype(np.uint32)
bc_fixed, fixed_nodes, fixed_dofs = wbm.bc_fixed
bc_fixed[5]

U_1 = wbm.hist.U_t[-1]
np.max(np.fabs(U_1))

8.089402997184195